In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from astropy.time import Time, TimezoneInfo
from statsmodels.tsa.stattools import adfuller

from lsst.sitcom import vandv
from lsst.ts.idl.enums import MTM1M3

from lsst.summit.utils.tmaUtils import TMAEventMaker, TMAState
from lsst.summit.utils.efdUtils import getEfdData, makeEfdClient

import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Select data from a given date
dayObs = 20230717
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)

# Get lists of slew and track events
slews = [e for e in events if e.type==TMAState.SLEWING]
client = eventMaker.client

print(f"{len(slews)} slews\nbegin: {slews[0].begin.iso}\nend: {slews[-1].end.iso}")

In [ ]:
slew_begin_utc=[]
slew_end_utc=[]
for slew in slews:
    slew_begin_utc.append(slew.begin.utc)
    slew_end_utc.append(slew.end.utc)

In [ ]:
begin_time=Time('2023-07-18 04:57:04')
end_time=Time('2023-07-18 05:02:04')

In [ ]:
df_mtmount_ele = getEfdData(client,'lsst.sal.MTMount.elevation', 
                            begin=begin_time,
                            end=end_time,
                            postPadding = postPadding)
df_mtmount_azi = getEfdData(client,'lsst.sal.MTMount.azimuth', 
                            begin=begin_time,
                            end=end_time, 
                            postPadding = postPadding)
df_ims = getEfdData(client, 'lsst.sal.MTM1M3.imsData', 
                    begin=begin_time,
                    end=end_time, 
                    postPadding = postPadding)

In [ ]:
def add_slew_ends(ax, timex, j):
    for i, slew in enumerate(slews):
        if (slew_end_utc[i].datetime > timex.min()) & (slew_end_utc[i].datetime < timex.max()):
            if j == 0:
                ax.axvline(slew_end_utc[i].datetime, c="k", zorder=1, ls="dashed", label="slew ends")
                j+=1
            else:
                ax.axvline(slew_end_utc[i].datetime, c="k", zorder=1, ls="dashed")
    return j

In [ ]:
fig, axs=plt.subplots(4, dpi=125, figsize=(10,5), sharex=True)

ax=axs[0]
ax.set_title(begin_time.iso + " to " + end_time.iso)
key="xPosition"
timex=Time(df_ims["timestamp"], format="unix_tai",scale="utc").datetime

ax.plot(timex, df_ims[key]*1e3, zorder=2, label=key)
j=0
j=add_slew_ends(ax,timex, j)
ax.legend(ncol=2)
ax.set(xlim=(timex.min(), timex.max()), 
       ylim=np.percentile(df_ims[key]*1e3,[1.5,98]),
       ylabel=key + "\nmm")


ax=axs[1]
key="yPosition"
ax.plot(timex, df_ims[key]*1e3, zorder=2, label=key, c="tab:orange")
j=add_slew_ends(ax,timex, j)
ax.legend()
ax.set(xlim=(timex.min(), timex.max()), 
       ylim=np.percentile(df_ims[key]*1e3,[1.5,98]),
       ylabel=key + "\nmm")


ax=axs[2]
key="zPosition"
ax.plot(timex, df_ims[key]*1e3, zorder=2, label=key, c="tab:green")
j=add_slew_ends(ax,timex, j)
ax.legend()
ax.set(xlim=(timex.min(), timex.max()), 
       ylim=np.percentile(df_ims[key]*1e3,[1.5,98]),
       ylabel=key + "\nmm")


ax=axs[3]
timex=Time(df_mtmount_ele["timestamp"], format="unix_tai",scale="utc").datetime
ax.plot(timex, df_mtmount_ele["actualPosition"], label="Position", zorder=2)
ax.legend(loc=2)
ax2=ax.twinx()
ax2.plot(timex, df_mtmount_ele["actualVelocity"], label="Velocity", c="tab:orange")
ax2.legend()
for i, slew in enumerate(slews):
    if (slew_end_utc[i].datetime > timex.min()) & (slew_end_utc[i].datetime < timex.max()):
        ax2.axvline(slew_end_utc[i].datetime, c="k", zorder=1, ls="dashed")
ax.set(xlabel="Time", ylabel="elevation")
ax2.set(ylabel="velocity deg/s")
ax.set_zorder(ax2.get_zorder()+1)
ax.set_frame_on(False)
plt.subplots_adjust(hspace=0)

In [ ]:
i=80
postPadding=60
slews[i].begin.iso

In [ ]:
df_mtmount_ele = getEfdData(client,'lsst.sal.MTMount.elevation', 
                            event=slews[i], 
                            postPadding = postPadding)
df_mtmount_azi = getEfdData(client,'lsst.sal.MTMount.azimuth', 
                            event=slews[i], 
                            postPadding = postPadding)
df_ims = getEfdData(client, 'lsst.sal.MTM1M3.imsData', 
                    event=slews[i], 
                    postPadding = postPadding)

In [ ]:
vtime

In [ ]:
meanval

In [ ]:
fig, ax=plt.subplots(dpi=125)
timex=Time(df_ims["timestamp"], format="unix_tai",scale="utc").datetime
ax.plot(timex, df_ims["zPosition"])
vtime=slews[i].end.utc
etime=slews[i+1].begin.utc
ax.axvline(vtime.datetime, c="k")
ax.axvline(etime.datetime, c="k", ls="dashed")
sel=(timex > vtime.datetime) & (timex < etime.datetime)
meanval=np.median(df_ims["zPosition"][sel])
ax2=ax.twinx()
ax2.plot(timex, df_ims["zPosition"].rolling(30).std(), c="tab:orange")
ax2.plot(timex, df_ims["zPosition"]-meanval, c="tab:green")

In [ ]:
slews[i].end